In [21]:
import pandas as pd
import numpy as np

In [22]:
pd.__version__

'2.3.3'

In [23]:
url = "https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv"
df = pd.read_csv(url)

In [24]:
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


In [25]:
df.shape

(1462, 9)

In [26]:
df.columns

Index(['lead_source', 'industry', 'number_of_courses_viewed', 'annual_income',
       'employment_status', 'location', 'interaction_count', 'lead_score',
       'converted'],
      dtype='object')

In [27]:
df.dtypes

lead_source                  object
industry                     object
number_of_courses_viewed      int64
annual_income               float64
employment_status            object
location                     object
interaction_count             int64
lead_score                  float64
converted                     int64
dtype: object

In [152]:
categorical = list(df.dtypes[df.dtypes == 'object'].index)

In [153]:
categorical

['lead_source', 'industry', 'employment_status', 'location']

In [154]:
numerical = ["number_of_courses_viewed", "annual_income", "interaction_count", "lead_score"]

In [155]:
numerical

['number_of_courses_viewed',
 'annual_income',
 'interaction_count',
 'lead_score']

In [156]:
df[categorical].isnull().sum()

lead_source          0
industry             0
employment_status    0
location             0
dtype: int64

In [55]:
df[categorical] = df[categorical].fillna("NA")

In [56]:
df[categorical].isnull().sum()

lead_source          0
industry             0
employment_status    0
location             0
dtype: int64

In [57]:
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NA,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NA,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


In [58]:
df[numerical].isnull().sum()

number_of_courses_viewed    0
annual_income               0
interaction_count           0
lead_score                  0
dtype: int64

In [59]:
df["annual_income"] = df["annual_income"].fillna(0.0)

In [60]:
df[numerical].isnull().sum()

number_of_courses_viewed    0
annual_income               0
interaction_count           0
lead_score                  0
dtype: int64

# Question 1 - Mode 

In [61]:
industry_mode = df["industry"].mode()

In [62]:
print(industry_mode)

0    retail
Name: industry, dtype: object


In [63]:
industry_mode[0]

'retail'

# Question 2 - Correlation matrix

In [64]:
df[numerical].corrwith(df.converted).abs()

number_of_courses_viewed    0.435914
annual_income               0.053131
interaction_count           0.374573
lead_score                  0.193673
dtype: float64

# Question 3 - Mutual Information score

In [65]:
from sklearn.model_selection import train_test_split

In [66]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)

In [67]:
len(df_train), len(df_val), len(df_test)

(876, 293, 293)

In [68]:
len(df_train) + len(df_val) + len(df_test)

1462

In [73]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [77]:
y_train = df_train.converted.values
y_val = df_val.converted.values
y_test = df_test.converted.values

In [80]:
del df_train["converted"]
del df_val["converted"]
del df_test["converted"]

In [84]:
from sklearn.metrics import mutual_info_score

In [85]:
categorical

['lead_source', 'industry', 'employment_status', 'location']

In [93]:
def mutual_info_churn_score(series):
    return round(mutual_info_score(series, y_train), 2)

In [94]:
mi = df_train[categorical].apply(mutual_info_churn_score)
mi.sort_values(ascending=False)

lead_source          0.04
industry             0.01
employment_status    0.01
location             0.00
dtype: float64

# Question 4 - Train a Logistic Regression

In [187]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression

In [188]:
dv = DictVectorizer(sparse=False)

In [189]:
train_dict = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

In [190]:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)

In [191]:
model.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,42
,solver,'liblinear'
,max_iter,1000
,multi_class,'deprecated'


In [192]:
val_dict = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dict)

In [193]:
y_pred = model.predict_proba(X_val)[:, 1]

In [194]:
converted_decision = (y_pred >= 0.5)

In [195]:
accuracy = (converted_decision == y_val).mean()
print(f"Accuracy: {round(accuracy, 2)}")

Accuracy: 0.7


In [196]:
accuracy_unrounded = (converted_decision == y_val).mean()
print(f"Overall accuracy without rounding: {accuracy_unrounded}")

Overall accuracy without rounding: 0.6996587030716723


# Question 5 - Feature Elimination

In [198]:
all_features = categorical + numerical
print(all_features)

['lead_source', 'industry', 'employment_status', 'location', 'number_of_courses_viewed', 'annual_income', 'interaction_count', 'lead_score']


In [199]:
for feature in all_features[:]:
    # Delete feature from original list of features
    all_features.remove(feature)
    print(f"{feature} has been removed from original list")

    # Create dictionary of rest of train dataset features and vectorize
    train_dict = df_train[all_features].to_dict(orient='records')
    X_train = dv.fit_transform(train_dict)

    # Create dictionary of rest of validation dataset features and vectorize
    val_dict = df_val[all_features].to_dict(orient='records')
    X_val = dv.transform(val_dict)

    # Fit model with train
    model.fit(X_train, y_train)

    # Predict on validation dataset
    y_pred = model.predict_proba(X_val)[:, 1]

    # Get accuracy without the feature that was removed
    converted_decision = (y_pred >= 0.5)
    print(f"Difference between the original accuracy and the accuracy without {feature} is: {accuracy_unrounded - (converted_decision == y_val).mean()}")

    if len(all_features) == 1:
        break

lead_source has been removed from original list
Difference between the original accuracy and the accuracy without lead_source is: -0.0034129692832765013
industry has been removed from original list
Difference between the original accuracy and the accuracy without industry is: -0.0034129692832765013
employment_status has been removed from original list
Difference between the original accuracy and the accuracy without employment_status is: 0.0068259385665528916
location has been removed from original list
Difference between the original accuracy and the accuracy without location is: 0.0068259385665528916
number_of_courses_viewed has been removed from original list
Difference between the original accuracy and the accuracy without number_of_courses_viewed is: 0.14334470989761094
annual_income has been removed from original list
Difference between the original accuracy and the accuracy without annual_income is: 0.023890784982935176
interaction_count has been removed from original list
Diffe

# Regularized Logistic Regression

In [203]:
dv = DictVectorizer(sparse=False)

In [204]:
train_dict = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

In [205]:
val_dict = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dict)

In [206]:
C = [0.01, 0.1, 1, 10, 100]

In [208]:
for item in C:
    model = LogisticRegression(solver='liblinear', C=item, max_iter=1000, random_state=42)
    model.fit(X_train, y_train)
    
    y_pred = model.predict_proba(X_val)[:, 1]
    
    converted_decision = (y_pred >= 0.5)
    accuracy = (converted_decision == y_val).mean()
    print(f"Accuracy for C with a value of {item}: {accuracy}")

Accuracy for C with a value of 0.01: 0.6996587030716723
Accuracy for C with a value of 0.1: 0.6996587030716723
Accuracy for C with a value of 1: 0.6996587030716723
Accuracy for C with a value of 10: 0.6996587030716723
Accuracy for C with a value of 100: 0.6996587030716723
